In [1]:
#import os, sys
#sys.path.insert(0, os.path.abspath('../src/models'))
import os
os.chdir('../../src/models')

In [2]:
import plotly.express as px
import plotly.graph_objects as go
import datetime

set3 = px.colors.qualitative.Set3

color_map_set3 = {
    'Lockdown':          set3[3],
    'Hard Quarantine':    set3[11],\
    'Light Quarantine':   set3[1],
    'Social Distancing':  'rgb(204, 245, 175)',
    'Unrestricted':        set3[6]     
}

def make_SIR_graph(data):
    d_counts = pd.DataFrame([pd.Series(d).value_counts() for d in data])
    d_counts.fillna(0, inplace=True)
    
    fig = go.Figure()
    x = list(range(len(d_counts[0])))

    fig.add_trace(go.Scatter(x=x, y=d_counts[-1]/len(G.nodes()), name='removed',line_color='green'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[0]/len(G.nodes()), name='susceptible', line_color='blue'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[1]/len(G.nodes()), name='exposed',line_color ='orange'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[2]/len(G.nodes()), name='infected', line_color = 'red'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[3]/len(G.nodes()), name='hospitalized', line_color = 'purple'))
    fig.update_layout(hovermode='x')
    fig.show()
    
    return d_counts

def make_beds_graph(data, actions, step_size, title, color_map):
    fig = go.Figure()
   
    x = pd.date_range(datetime.date(2020, 2, 24), periods=len(data)+2)

    fig.add_trace(go.Scatter(x=x, y=data[3]/55e3, name='hospitalized', line_color = 'royalblue',
                            line=dict(width=3)))
    fig.add_trace(go.Scatter(x=x, y=data[1]/55e3, name='exposed', line_color = 'firebrick',
                            line=dict(width=3)))
    fig.add_trace(go.Scatter(x=x, y=len(data)*[0.0025], name='capacity', line_color = 'black',
                            line=dict(dash='dash', width = 2)))
    fig.update_layout(
        shapes=[
            dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=x[step_size*i],
                y0=0,
                x1=x[step_size*(i+1)-1],
                y1=1,
                fillcolor=a,
                opacity=0.5,
                layer="below",
                line_width=0,
            ) for i,a in enumerate(actions)] 
    )

    for k,v in color_map.items():
        fig.add_trace(go.Bar(x=[None], y=[None], marker=dict(color=v), name = k))

    fig.update_layout(coloraxis = {'colorscale':'deep'}, xaxis={'showgrid': False},
                      yaxis = {'showgrid': False},
                      showlegend=True, title = title, hovermode="x")

    fig.show()

In [3]:
from policies import policies, costs

In [4]:
from tqdm import tqdm
import numpy as np
import pandas as pd

In [5]:
import numpy as np
import networkx as nx
from tqdm import tqdm
from disease_states import states_dict
from patient_evolution import susceptible_to_exposed, change_state
from functools import partial
from policies import policies
from collections import defaultdict

print('Loading Graph...',  end='')
G = nx.read_gpickle('../../data/processed/SP_multiGraph_Job_Edu_Level.gpickle')
print('Done!')

prhome = 0.06
p_r = {
    'home'    :  prhome,
    'neighbor':  .05*prhome,
    'work'    :  .05*prhome,
    'school'  :  .2*prhome,
}

def init_infection(pct=.0001):
    """
    Given a Graph G, infects pct% of population and set the
    remainder as susceptible. This is considered day 0.

    Args:
        pct (float): percentage of people initially infected.

    Returns:
        new_matrix (np.array): 2D Array  arrays of id, state, day of infection
            and current state duration of population.
    """

    global G

    size = int(len(G.nodes) * pct)
    infected = list(np.random.choice(G.nodes(), size=size, replace=False))
    infected = [x for x in infected]

    pop_matrix = np.array([[node, states_dict['susceptible'],
                            -1, -1, data['age']]
                          for node, data in G.nodes(data=True)])
    
    contacts_infected = defaultdict(int)

    matrix_change = pop_matrix[np.isin(pop_matrix[:, 0], infected)]

    matrix_keep = pop_matrix[~np.isin(pop_matrix[:, 0], infected)]

    matrix_change = np.apply_along_axis(susceptible_to_exposed,
                                        1, matrix_change, day=0)

    new_matrix = np.concatenate((matrix_keep, matrix_change))
    assert new_matrix.shape == pop_matrix.shape

    return new_matrix, contacts_infected


def expose_population(pop_matrix, exposed, day):
    """
    Receives the population matrix, an array containing ids of newly
    exposed individuals and the current simulation day

    Args:
        pop_matrix (np.array): 2D Array  arrays of id, state, day of infection
            and current state duration of population.

        exposed (list): list of newly exposed people id
        day (int): current simulation day

    Returns:
        new_matrix (np.array): The population matrix with the
        newly exposed people exposed.
    Raises:
        ValueError: If shape of starting matrix is different from final matrix
    """
    matrix_change = pop_matrix[np.isin(pop_matrix[:, 0], exposed)]
    matrix_keep = pop_matrix[~np.isin(pop_matrix[:, 0], exposed)]
    matrix_change = np.apply_along_axis(susceptible_to_exposed,
                                        1, matrix_change, day=day)

    new_matrix = np.concatenate((matrix_keep, matrix_change))

    if new_matrix.shape != pop_matrix.shape:
        raise ValueError("Input and output matrix shapes are different")
    return new_matrix


def lambda_leak_expose(pop_matrix, day, lambda_leak=.0001):
    """
    Receives the population matrix, the current day and the leak factor.
    Chooses at random a lambda_leak percentage of the population to expose

    Args:
        pop_matrix (np.array): 2D Array  arrays of id, state, day of infection
            and current state duration of population.

        lambda_leak (float): the percentage of the population to expose

        day: the current day of simulation

    Returns:
        new_matrix (np.array): The population matrix with the
                               newly exposed people.
    Raises:
        ValueError: If shape of starting matrix is different from final matrix
    """
    size = int(pop_matrix.shape[0]*lambda_leak)

    susceptible = pop_matrix[np.where(pop_matrix[:, 1]
                                      == states_dict['susceptible'])][:, 0]

    exposed = np.random.choice(susceptible, size=size, replace=False)

    if len(exposed) == 0:
        return pop_matrix

    new_matrix = expose_population(pop_matrix, exposed, day)

    if new_matrix.shape != pop_matrix.shape:
        raise ValueError("Input and output matrix shapes are different")

    return new_matrix


def spread_through_contacts(spreader, restrictions):
    """
    An infected person, or a spredear, infects each of
    its contacts with chance equal to
    np.random.random() < p_r[r] * (1 - restrictions[r]).
    Returns an array of all the people successfully infected by the spreader.

    Args:
        spreader (int): id of the infected person that is
                        spreading the disease.

        restrictions (dictionary): a dictionary with a value between
                                   zero and one for each type of relation

    Returns:
        infected (list): List of all the people infected by spreader.
    """
    global G, p_r
    spreader = spreader
    contacts = [[y, v['edge_type']] for x, y, v
                in G.edges(spreader, data=True)]

    infected = [y for r in restrictions.keys() for y, v in contacts
                if v == r
                and np.random.random() < p_r[r] * (1 - restrictions[r])]

    return infected


def update_population(pop_matrix):
    """
    Receives the population matrix and progress the infections
    for all people. The state duration is decremented
    and for those whom it reaches zero, they transition to the next state

    Args:
        pop_matrix (np.array): 2D Array  arrays of id, state, day of infection
            and current state duration of population.

    Returns:
        new_matrix (np.array): The population matrix with the updated status.
    Raises:
        ValueError: If shape of starting matrix is different from final matrix
    """
    matrix_keep = pop_matrix[np.isin(pop_matrix[:, 1],
                                     [states_dict['susceptible'],
                                     states_dict['removed']]
                                     )]
    matrix_change = pop_matrix[~np.isin(pop_matrix[:, 1],
                                        [states_dict['susceptible'],
                                        states_dict['removed']]
                                        )]

    matrix_change[:, 3] = matrix_change[:, 3].astype(int) - 1
    matrix_no_change = matrix_change[matrix_change[:, 3].astype(int) > 0]
    matrix_change = matrix_change[matrix_change[:, 3].astype(int) == 0]

    if matrix_change.shape[0] > 0:
        matrix_change = np.apply_along_axis(change_state, 1, matrix_change)

    new_matrix = np.concatenate((matrix_keep, matrix_change, matrix_no_change))

    if new_matrix.shape != pop_matrix.shape:
        raise ValueError("Input and output matrix shapes are different")
    return new_matrix



def spread_infection(pop_matrix, restrictions, day, contacts_infected=None):
    """
    Receives the population matrix, the restrictions dictionary and the
    current day. The disease spreads throught the relations in the graph:
    each infected person has a chance to infect a susceptible contact with
    it has an edge in the graph, conditioned to the current restrictions,
    the p_r of each relation.

    Args:
        pop_matrix (np.array): 2D Array  arrays of id, state, day of infection
            and current state duration of population.

        restrictions (dictionary): a dictionary with a value between
        zero and one for each type of relation

        day: the current day of simulation

    Returns:
        new_matrix (np.array): The population matrix with the newly
                                exposed people.
    Raises:
        ValueError: If shape of starting matrix is different from final matrix
    """
    global G
    mask = np.where(pop_matrix[:, 1] == states_dict['infected'])
    currently_infected = pop_matrix[mask][:, 0]

    if currently_infected.shape[0] == 0:
        return pop_matrix, contacts_infected
        
    exposed = []
    for spreader in currently_infected:
        exp = spread_through_contacts(spreader, restrictions)
        exp = [e for e in exp if e not in exposed]
        if contacts_infected is not None:
            contacts_infected[spreader] += len(exp)
        exposed.extend(exp)
        
    exposed = np.unique(exposed)

    mask = np.isin(pop_matrix[:, 0], exposed)
    susceptible = np.isin(pop_matrix[np.array(mask)][:, 1],
                          states_dict['susceptible'])
    exposed = pop_matrix[np.array(mask)][:, 0][susceptible]

    if len(exposed) == 0:
        return pop_matrix, contacts_infected

    new_matrix = expose_population(pop_matrix, exposed, day)

    if new_matrix.shape != pop_matrix.shape:
        raise ValueError("Input and output matrix shapes are different")


    return new_matrix, contacts_infected


def main(policy='Unrestricted', days=500):
    """
    Receives the policy to be used during the simulation and for how many days
    the simulation should run for. The policy should be a Key in the policies
    dict inside policies.py.

    One full step consists of:
        Spreading the infection
        Exposing through leakage
        Updating the disease evolution of the population

    Args:
        pop_matrix (string): The name of a policy that exists is policies.policies.

        days (int): For how long should the policy run.


    Returns: data (np.array): An array of arrays containing the status of 
    the population at each time step.
    
    """

    pop_matrix, contacts_infected = init_infection(.0001)

    data = []

    # restrictions={'work':0, 'school': 0, 'home':0, 'neighbor':0}
    restrictions = policies[policy]
    print(restrictions)

    for day in tqdm(range(1, days)):
        # if less than 90% already recovered, break simulation
        if (pop_matrix[np.where(pop_matrix[:, 1] == -1)].shape[0] > pop_matrix.shape[0]*.9):
            break

        pop_matrix, contacts_infected = spread_infection(pop_matrix, restrictions, day, contacts_infected)
        pop_matrix = lambda_leak_expose(pop_matrix, day)
        pop_matrix = update_population(pop_matrix)

        data.append(np.array(sorted(pop_matrix, key=lambda x: x[0]))[:, 1])

    return data, contacts_infected, pop_matrix

Loading Graph...Done!


In [ ]:
prhome = 0.06

p_r = {
    'home'    :  prhome,
    'neighbor':  .1*prhome,
    'work'    :  .1*prhome,
    'school'  :  .15*prhome,
}

def one_full_sim():
    pop_matrix, contacts_infected = init_infection(.0001)

    data = []
    days = 300

    #restrictions = policies[policy]

    for day in range(1, days):
        # if less than 90% already recovered, break simulation
#        if (pop_matrix[np.where(pop_matrix[:, 1] == 0)].shape[0]
#                < pop_matrix.shape[0]*.97):
#            break
        if (pop_matrix[np.where(pop_matrix[:, 1] == -1)].shape[0] > pop_matrix.shape[0]*.9):
            break

        restrictions = policies['Unrestricted']
        #if day <= 15:
        #    restrictions = policies['Unrestricted']
        #else : 
        #    restrictions = policies['Light Quarantine']
        #else: 
        #    restrictions = policies['Social Distancing']

        pop_matrix, contacts_infected = spread_infection(pop_matrix, restrictions, day, contacts_infected)
        pop_matrix = lambda_leak_expose(pop_matrix, day)
        pop_matrix = update_population(pop_matrix)

        data.append(np.array(sorted(pop_matrix, key=lambda x: x[0]))[:, 1])
        
    return pop_matrix, contacts_infected, data

In [15]:
data = [one_full_sim() for i in tqdm(range(100))]

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:25<00:00,  2.05s/it]


In [17]:
fig = go.Figure()
for d in data:
    r = []
    for day in range(5, 15):
        infected = d[0][d[0][:,2] == day][:, 0]
        r.append(np.mean([d[1][i] for i in infected]))
    fig.add_trace(go.Scatter(x=list(range(len(r))), y=r, mode='lines', line=dict(color='red'), opacity=0.3, showlegend=False))

mean_line = []
for day in range(5, 15):
    r = []
    for d in data:
        infected = d[0][d[0][:,2] == day][:, 0]
        r.extend([d[1][i] for i in infected])
    mean_line.append(np.mean(r))

fig.add_trace(go.Scatter(x=list(range(len(mean_line))), y=mean_line, mode='markers+lines', 
                         line=dict(color='black'), opacity=0.7, name='Average'))

fig.write_image('light_quearentine_rt.png', scale=3)
fig.show()

In [25]:
df = make_SIR_graph(one_full_sim()[2])
#actions_3_colors = list(map(color_map_set3.get,  actions))
#make_beds_graph(df, actions_3_colors, 7, 'Sim', color_map_set3)

In [ ]:
fig = go.Figure()
for d in data:
    r = []
    for day in range(5, 15):
        infected = d[0][d[0][:,2] == day][:, 0]
        r.append(np.mean([d[1][i] for i in infected]))
    fig.add_trace(go.Scatter(x=list(range(len(r))), y=r, mode='lines', line=dict(color='red'), opacity=0.3))

mean_line = []
for day in range(5, 15):
    r = []
    for d in data:
        infected = d[0][d[0][:,2] == day][:, 0]
        r.append(np.mean([d[1][i] for i in infected]))
    mean_line.append(np.mean(r))

fig.add_trace(go.Scatter(x=list(range(len(mean_line))), y=mean_line, mode='markers+lines', 
                         line=dict(color='black'), opacity=0.7))

fig.show()

In [ ]:
r = []
for day in range(5, 15):
    infected = pop_matrix[pop_matrix[:,2] == day][:, 0]
    r.append(np.mean([contacts_infected[i] for i in infected]))
fig.add_trace(go.Scatter(x=list(range(len(r))), y=r, mode='lines+markers'))
fig.add_trace(go.Scatter(x=list(range(len(r))), y=[np.mean(r)]*len(r), line=dict(dash='dash')))

fig.show()

In [ ]:
rolling_mean = pd.Series(r).rolling(3).mean()
fig = go.Figure()
ranges = [0,len(df)]
colors = ['gray']#, 'yellow']#, 'green']
for i in range(len(ranges)-1):
    rll = rolling_mean[ranges[i]:ranges[i+1]]
    fig.add_trace(go.Scatter(x=list(range(ranges[i], ranges[i+1])), y=rll, 
                             mode='markers+lines', marker=dict(color=colors[i]), ))
    fig.add_trace(go.Scatter(x=list(range(ranges[i], ranges[i+1])), y=[np.mean(rll)]*len(rll), 
                             marker=dict(color='black'),line=dict(dash='dash') ))
fig.show()
